In [8]:
#!pip install transformers==4.26

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 127.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 198.1 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.21.1
    Uninstalling tokenizers-0.21.1:
      Successfully uninstalled tokenizers-0.21.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.51.3
    Uninstalling transformers-4.51.3:
      Successfully uninstalled transformers-4.51.3


In [1]:
import sagemaker
import boto3
import json

try:
    role = sagemaker.get_execution_role()
except ValueError:
    iam = boto3.client('iam')
    role = iam.get_role(RoleName='SageMaker-MLengineer')['Role']['Arn']

print(f"sagemaker role arn: {role}")

/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/pydantic/_internal/_fields.py:172: UserWarning: Field name "json" in "MonitoringDatasetFormat" shadows an attribute in parent "Base"
  warnings.warn(


[05/16/25 13:19:58] INFO     Found credentials from IAM Role:                                   ]8;id=730212;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=109498;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/botocore/credentials.py#1132\1132]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


                    INFO     Found credentials from IAM Role:                                   ]8;id=119703;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=624295;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/botocore/credentials.py#1132\1132]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

sagemaker role arn: arn:aws:iam::311410995876:role/service-role/SageMaker-MLengineer


In [2]:
role

'arn:aws:iam::311410995876:role/service-role/SageMaker-MLengineer'

In [9]:
from transformers import AutoModelForQuestionAnswering, AutoTokenizer
import os

model_id = "distilbert-base-uncased-distilled-squad"
model_dir = "model"
os.makedirs(model_dir, exist_ok=True)

try:
    model = AutoModelForQuestionAnswering.from_pretrained(model_id)
    tokenizer = AutoTokenizer.from_pretrained(model_id)
    model.save_pretrained(model_dir)
    tokenizer.save_pretrained(model_dir)
    print(f"Model and tokenizer saved to {model_dir}")
except Exception as e:
    print(f"Error downloading model: {str(e)}")
    raise

config.json:   0%|          | 0.00/451 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


[05/15/25 11:01:59] WARNING  Xet Storage is enabled for this repo, but the 'hf_xet' package   ]8;id=178090;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/huggingface_hub/file_download.py\file_download.py]8;;\:]8;id=116117;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/huggingface_hub/file_download.py#1717\1717]8;;\
                             is not installed. Falling back to regular HTTP download. For                          
                             better performance, install the package with: `pip install                            
                             huggingface_hub` or `pip install hf_xet`                                              

model.safetensors:   0%|          | 0.00/265M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Model and tokenizer saved to model


In [10]:
!tar -czf model.tar.gz -C model .

In [5]:
import boto3
from botocore.exceptions import ClientError
import os

# Configuration
bucket_name = 'deploy-huggingface-model'
file_path = 'model.tar.gz'  # Local path to the file
s3_key = 'model.tar.gz'  # S3 path (e.g., model.tar.gz in the bucket root)
region = 'eu-north-1'

# Initialize S3 client
s3_client = boto3.client('s3', region_name=region)

def upload_file_to_s3(local_path, bucket, s3_path):
    """
    Upload a file to an S3 bucket.
    
    Args:
        local_path (str): Path to the local file
        bucket (str): S3 bucket name
        s3_path (str): S3 key (path in the bucket)
    
    Returns:
        bool: True if upload succeeds, False otherwise
    """
    try:
        # Check if the local file exists
        if not os.path.exists(local_path):
            print(f"Error: File {local_path} does not exist")
            return False
        
        # Upload the file
        print(f"Uploading {local_path} to s3://{bucket}/{s3_path}")
        s3_client.upload_file(local_path, bucket, s3_path)
        print(f"Successfully uploaded {local_path} to s3://{bucket}/{s3_path}")
        return True
    
    except ClientError as e:
        error_code = e.response['Error']['Code']
        error_message = e.response['Error']['Message']
        print(f"Error uploading file: {error_code} - {error_message}")
        return False
    except Exception as e:
        print(f"Unexpected error: {str(e)}")
        return False

def verify_file_in_s3(bucket, s3_path):
    """
    Verify if the file exists in the S3 bucket.
    
    Args:
        bucket (str): S3 bucket name
        s3_path (str): S3 key
    
    Returns:
        bool: True if file exists, False otherwise
    """
    try:
        s3_client.head_object(Bucket=bucket, Key=s3_path)
        print(f"Verified: File s3://{bucket}/{s3_path} exists")
        return True
    except ClientError as e:
        if e.response['Error']['Code'] == '404':
            print(f"File s3://{bucket}/{s3_path} does not exist")
        else:
            print(f"Error verifying file: {e.response['Error']['Code']} - {e.response['Error']['Message']}")
        return False
    except Exception as e:
        print(f"Unexpected error verifying file: {str(e)}")
        return False

# Main execution
if __name__ == "__main__":
    # Upload the file
    if upload_file_to_s3(file_path, bucket_name, s3_key):
        # Verify the upload
        verify_file_in_s3(bucket_name, s3_key)
    else:
        print("Upload failed. Please check the error messages above.")

[05/15/25 11:20:07] INFO     Found credentials from IAM Role:                                   ]8;id=608718;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=431264;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/botocore/credentials.py#1132\1132]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

Uploading model.tar.gz to s3://deploy-huggingface-model/model.tar.gz
Successfully uploaded model.tar.gz to s3://deploy-huggingface-model/model.tar.gz
Verified: File s3://deploy-huggingface-model/model.tar.gz exists


In [3]:
from sagemaker.huggingface import HuggingFaceModel

# Recreate and redeploy with correct configuration
hub = {
    'HF_MODEL_ID': 'distilbert-base-uncased',
    'HF_TASK': 'text-classification'
}

# Create Hugging Face Model Class
try:
    huggingface_model = HuggingFaceModel(
        role=role,
        env=hub,
        transformers_version="4.26",
        pytorch_version="1.13",
        py_version="py39",
    #    model_data='s3://deploy-huggingface-model/model.tar.gz'
    )
    print("HuggingFaceModel created successfully")
except Exception as e:
    print(f"Error creating HuggingFaceModel: {str(e)}")
    raise

[05/16/25 13:20:23] INFO     Found credentials from IAM Role:                                   ]8;id=774415;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=163001;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/botocore/credentials.py#1132\1132]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

HuggingFaceModel created successfully


In [4]:

predictor = huggingface_model.deploy(
    initial_instance_count=1,
    instance_type="ml.m5.xlarge",
    endpoint_name="huggingface-distilbert-endpoint2"
)

[05/16/25 13:20:34] INFO     Creating model with name:                                              ]8;id=312518;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=478771;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/sagemaker/session.py#4094\4094]8;;\
                             huggingface-pytorch-inference-2025-05-16-13-20-34-052                                 

                    INFO     Creating endpoint-config with name huggingface-distilbert-endpoint2    ]8;id=927329;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=832047;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/sagemaker/session.py#6019\6019]8;;\

                    INFO     Creating endpoint with name huggingface-distilbert-endpoint2           ]8;id=385946;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=558687;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/sagemaker/session.py#4841\4841]8;;\

--------!

In [5]:
# example request, you always need to define "inputs"
data = {
   "inputs": "The new Hugging Face SageMaker DLC makes it super easy to deploy models in production. I love it!"
}

# request
predictor.predict(data)

[{'label': 'LABEL_1', 'score': 0.5334737300872803}]

In [6]:
predictor.delete_endpoint()

[05/16/25 13:33:45] INFO     Deleting endpoint configuration with name:                             ]8;id=272327;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=649481;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/sagemaker/session.py#4995\4995]8;;\
                             huggingface-distilbert-endpoint2                                                      

                    INFO     Deleting endpoint with name: huggingface-distilbert-endpoint2          ]8;id=151194;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=583456;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/sagemaker/session.py#4985\4985]8;;\